In [988]:
# === norvig's code
# steal from it without shame

# import re
# from collections import Counter

# def words(text): return re.findall(r'\w+', text.lower())

# WORDS = Counter(words(open('big.txt').read()))

# def P(word, N=sum(WORDS.values())): 
#     "Probability of `word`."
#     return WORDS[word] / N

# def correction(word): 
#     "Most probable spelling correction for word."
#     return max(candidates(word), key=P)

# def candidates(word): 
#     "Generate possible spelling corrections for word."
#     return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

# def known(words): 
#     "The subset of `words` that appear in the dictionary of WORDS."
#     return set(w for w in words if w in WORDS)

# def edits1(word):
#     "All edits that are one edit away from `word`."
#     letters    = 'abcdefghijklmnopqrstuvwxyz'
#     splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
#     deletes    = [L + R[1:]               for L, R in splits if R]
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
#     inserts    = [L + c + R               for L, R in splits for c in letters]
#     return set(deletes + transposes + replaces + inserts)

# def edits2(word): 
#     "All edits that are two edits away from `word`."
#     return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [989]:
# flatten a list of two tuples
def lst_flatten(lst):
    ret = []
    for a,b in lst:
        ret.append(a)
        ret.append(b)
    return ret


In [990]:

yuge_data = ""

cnt = 1
dataset_lst = ["yuge_data.txt", "internet-en.num.txt", "en.txt"]
for d in dataset_lst:
    data_import = open(d).read()
    yuge_data += data_import
    print(">>> importing  dataset #" + str(cnt))
    print(data_import[:128])
    print()

    cnt += 1


>>> importing  dataset #1
The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in our series by Sir Arthur Cona

>>> importing  dataset #2
The frequency distribution for attribute 'lemma' in corpus 'internet-en'
For more information visit http://corpus.leeds.ac.uk/li

>>> importing  dataset #3
you 6281002
i 5685306
the 4768490
to 3453407
a 3048287
it 2879962
and 2127187
that 2030642
of 1847884
in 1554103
what 1497273
is



In [991]:
import re
from collections import Counter

def extract_words(src):
    # allow upper case words
    return re.findall(r'\b\w+\b', src)
#     return re.findall(r'\b\w+\b', src.lower())


In [992]:
# a regex test
# tst_str = "The Project Gutenberg EBook of The Adventures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in our series by Sir Arthur Conan Doyle)\n\nCopyright laws are changing all over the world. Be sure to check the\ncopyright laws for your country before downloadin"
# re.findall(r'\b\w+\b',tst_str)

extract_words(yuge_data)
WORD_IDX = Counter(extract_words(yuge_data))

# retrieve the 7 most common words
WORD_IDX.most_common(7)

[('1', 210511),
 ('the', 72296),
 ('2', 64161),
 ('of', 39717),
 ('and', 36894),
 ('3', 31024),
 ('to', 28269)]

In [993]:
# how likely is a given word?

# get the sum of all words
ALL_WORD_CNT = sum(WORD_IDX.values())

# get the number of occurrences 
WORD_IDX.values()

# probability function
def ProbabilityForWord(word):
    return WORD_IDX[word]/ALL_WORD_CNT


In [994]:
# building our candidate model
# (this boils down to generating possible words 
#  based on simple string manipulation)


# [print(x) for x in s]
# print()
# [print(x) for x in d]
# print()
# [print(x) for x in t]
# print()
# [print(x) for x in r]
# print()
# [print(x) for x in i]
# print()

def candidates_1(word):
    # "".join([chr(c) for c in range(65, 91)])
    alphab = 'abcdefghijklmnopqrstuvwxyz'+'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    # edits formed by inserting a space
    s = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    # edits formed by deleting a character
    d = [L + R[1:] for L, R in s if R]

    # edits formed by swapping the i'th and (i+1)'th characters
    # where i < len(word) - 1
    t = [L + R[1] + R[0] + R[2:] for L, R in s if len(R)>1] 
    # TODO: consider adding transposes of a longer length 
    # (i.e., three or four letter transposes)

    # edits formed by substituting a random character
    # (similar to: "edits formed by inserting a space" as shown above)
    r = []
    for L, R in s:
        if R:
            for c in alphab:
                r.append( L + c + R[1:] )

    # edits formed by inserting a random char
    i = [L + c + R for L, R in s for c in alphab]

    # dont exclude split words
    # consider them as candidates
    s = lst_flatten(s)

    # return all: splits, deletes, edits, replacements, and insertions
    return set(s + d + t + r + i)

'''
# TODO: define a recursive candidate function
# (this kinda like DFS)
def candidates_n(word, n):
    if (n == 1):
        candidates_1(word)
    else:
        for a in candidates_1(word):
            for b in candidates

# tests
# candidates_1("pineapple")
# candidates_n("pineapple", 1)
# candidates_n("q", 1)
'''

# TODO: fix this (not working for some reason)
# def candidates_2(word):
#     "All edits that are two edits away from `word`."
#     return (e2 for e1 in candidates_1(word) for e2 in candidates_1(e1))

# candidates_2("pineapple")


'\n# TODO: define a recursive candidate function\n# (this kinda like DFS)\ndef candidates_n(word, n):\n    if (n == 1):\n        candidates_1(word)\n    else:\n        for a in candidates_1(word):\n            for b in candidates\n\n# tests\n# candidates_1("pineapple")\n# candidates_n("pineapple", 1)\n# candidates_n("q", 1)\n'

In [995]:
# can a given word be found in the "dictionary"?
def only_words_in_dict(words):
    assert( type(words) == set )
    return set(w for w in words if WORD_IDX[w] > 0)


In [996]:
# === unit/regression tests

print("prob(the)", ProbabilityForWord('the'))
print("prob(pineapple)", ProbabilityForWord("pineapple"))
print("pineapple", WORD_IDX["pineapple"])
print("all words", ALL_WORD_CNT)

# A = 65
# Z = 90
for upper_case_letter in range(65, 91):
    assert( ProbabilityForWord(upper_case_letter) < 1 )
    assert( ProbabilityForWord(upper_case_letter) >= 0 )
    assert( WORD_IDX[upper_case_letter] < ALL_WORD_CNT )

# a = 97
# z = 122
for lower_case_letter in range(97, 123):
    assert( ProbabilityForWord(lower_case_letter) < 1 )
    assert( ProbabilityForWord(lower_case_letter) >= 0 )
    assert( WORD_IDX[lower_case_letter] < ALL_WORD_CNT )

common_words = ["the", "i", "I", "am"]
for c in common_words:
    assert( ProbabilityForWord(c) < 1)

# generate all possible edits for a word
tst_word = "pineapple"
[(a, WORD_IDX[a]) for a in only_words_in_dict(candidates_1(tst_word))]

prob(the) 0.03427271688648087
prob(pineapple) 1.422183117453837e-06
pineapple 3
all words 2109433


[('apple', 13),
 ('neapple', 1),
 ('p', 43),
 ('pineaple', 1),
 ('le', 45),
 ('pineapples', 3),
 ('pineappple', 1),
 ('ple', 1),
 ('pi', 2),
 ('pin', 20),
 ('pineapple', 3),
 ('e', 30),
 ('pple', 1),
 ('pine', 11)]